# Data Cleaning Challenge

Author:Nutan Mandale

Mentor:Vincent Ramdhanie

This is the data cleaning exercise for the Wellcome trust journal data.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns

In [2]:
trust = pd.read_csv("./WELLCOME_APCspend2013_forThinkful.csv",encoding='latin')
df = pd.DataFrame(trust)
trust.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


Taking a look at data we can see there are many inconsistancies in the data.In the column of PMID/PMCID we can see there are rows without data ,inconsistent format of the ID.In the Publisher column we see the publisher is abbreviated as well as in full name.
The Journal title column have similar inconsistancies as that of Publisher column.The cost column have the currency symbol which needs to be removed inorder to get the cost as float number.

In [3]:
#Rename the cost column
df = df.rename(columns={'COST (£) charged to Wellcome (inc VAT when charged)':'Cost'})


In [4]:
#Extract the numeric money value from cost field.
#Cost includes currency symbol
def cleanCost(cost):
    nums = re.findall(r'\d+\.*\d*', cost)
    return float(nums[0])

In [5]:
#df3 contains a data frame with the cleaned costs
df3 = pd.DataFrame({'Cost': df['Cost'].apply(cleanCost)})
df3.max()

Cost    999999.0
dtype: float64

The maximum value of the cost 999999.0 seems to be unrealistic. These outliers needs to be corrected.

In [6]:
def outlier_correction(cost, dataf, threshold):
    result = cost
    if cost >= threshold:
        result = dataf.quantile(.9).values[0]
    return result


In [7]:
New_Cost = pd.DataFrame({'Cost':df3['Cost'].apply(outlier_correction, args=(df3, 999998))})

In [8]:
def publisher(name):
    p = {
        'AMERCIAN CHEMICAL SOCIETY': 'ACS',
        'ACS':'ACS',
        'AMERICAN CHEMICAL SOCIETY':'ACS',
        'ACS (Amercian Chemical Society) Publications':'ACS',
        'ACS Publications':'ACS',
        'American Chemical Society':'ACS',
        'American Chemical Society Publications':'ACS',
        'American Soc for Biochemistry and Molecular Biology':'ASBMB',
        'American Society for Biochemistry and Molecular Biology':'ASBMB',
        'ASM (American Society for Microbiology)':'ASM',
        'American Society for Microbiology':'ASM',
        'ASBMB Cadmus':'ASBMB',
        'ASBMB/Cenveo Publisher Services':'ASMB',
        'ASBMC /CENVEO':'ASMB',
        'BioMed Central ':'BMC',
        'BioMed Central Limited':'BMC',
        'BioMed Central Ltd':'BMC',
        'BMJ Group':'BMJ',
        'BMJ Journals':'BMJ',
        'BMJ Publishing Group':'BMJ',
        'BMJ PUBLISHING GROUP':'BMJ',
        'BMJ Publishing Group Ltd':'BMJ',
        'BMJ Publishing Group Ltd & British Thoracic Society':'BMJ',
        'British Medical Journal':'BMJ',
        'CADMUS JOURNAL SERVICE ':'Cadmus',
        'Cadmus Journal Services':'Cadmus',
        'Cambridge Univ Press':'Cambridge University Press',
        'Cold Spring Habour Press':'Cold Spring Harbor',
        'Cold Spring Harbor Laboratory Press':'Cold Spring Harbor',
        'Cold Spring Harbor Publications':'Cold Spring Harbor',
        'Elseveier Science':'Elsevier',
        'Elsevier (Cell Press)':'Elsevier ',
        'ELSEVIER':'Elsevier ',
        'Elsevier / Cell Science':'Elsevier',
        'Elsevier B.V.':'Elsevier',
        'Elsevier Ltd':'Elsevier',
        'Elsevier/Cell Press':'Elsevier',
        'Federation of American Societies for Experimental Biology':'FASEB',
        'Frontiers Media':'Frontiers',
        'Frontiers Media SA':'Frontiers',
        'Frontiers Research Foundation':'Frontiers',
        'Future Medicine Ltd':'Future Medicine',
        'Hindawi Publishing Corporation':'Hindawi',
        'Impact Journals LLC':'Impact Journals',
        'JOHN WILEY & SONS':'John Wiley',
        'John Wiley & Sons':'John Wiley',
        'John Wiley & Sons Inc':'John Wiley',
        'John Wiley & Sons Ltd':'John Wiley',
        'John Wiley and Sons':'John Wiley',
        'John Wiley and Sons Ltd':'John Wiley',
        'KARGER':'Karger',
        'MARY ANN LIEBERT INC':'Mary Ann Liebert',
        'Mary Ann Liebert, Inc. Publishers':'Mary Ann Liebert',
        'My JOVE corporation':'MY JOVE CORP',
        'MYJoVE Corporation':'MY JOVE CORP',
        'National Academy of Sciences of the United States of America':'National Academy of Sciences',
        'National Academy of Sciences USA':'National Academy of Sciences',
        'Nature ':'Nature Publishing Group',
        'Nature PG':'Nature Publishing Group',
        'Nature Publishing':'Nature Publishing Group',
        'NATURE PUBLISHING GROUP LTD':'Nature Publishing Group',
        'NPG':'Nature Publishing Group',
        'Oxford Journals':'OUP',
        'Oxford Journals (OUP)':'OUP',
        'Oxford Univ Press':'OUP',
        'Oxford University Press':'OUP',
        'OXFORD UNIVERSITY PRESS':'OUP',
        'PLoS (Public Library of Science)':'PLOS',
        'Plos':'PLOS',
        'PLoS Public Library of Science':'PLOS',
        'PNAS Author Publication':'PNAS',
        'PORTLAND PRESS LTD':'Portland Press',
        'Portland Press Ltd':'Portland Press',
        'Royal Society':'RSC',
        'Royal Society of Chemistry':'RSC',
        'RSC Publishing':'RSC',
        'Sage':'SAGE',
        'Sage Publications':'SAGE',
        'SAGE Publications':'SAGE',
        'Sage Publications Inc':'SAGE',
        'Sage Publications Ltd':'SAGE',
        'Sage Publishers':'SAGE',
        'Sage Publishing':'SAGE',
        'Springer - Verlag GMBH':'Springer',
        'Springer Science + Business Media':'Springer',
        'Springer Verlag':'Springer',
        'SPRINGER-VERLAG GMBH':'Springer',
        'Springer-Verlag GMBH & Ci':'Springer',
        'Springer-Verlag GmbH, Heidelberger Platz 3, D-14197 Berlin':'Springer',
        'Taylor & Francis':'T&F',
        'Taylor & Francis Journals ':'T&F',
        'The company of Biolgists':'THE COMPANY OF BIOLOGISTS',
        'The Company of Biologists':'THE COMPANY OF BIOLOGISTS',
        'The Company of Biologists Ltd':'THE COMPANY OF BIOLOGISTS',
        'The Endocrine Society':'THE ENDOCRINE SOCIETY',
        'The royal Society':'The Royal Society',
        'Wiley & Son':'Wiley',
        'Wiley Blackwell':'Wiley',
        'Blackwell Publishing Ltd/Wiley':'Wiley',
        'Wiley Online Library':'Wiley',
        'Wiley Subscription Services':'Wiley',
        'Wiley Subscription Services  Inc.':'Wiley',
        'Wiley VCH':'Wiley',
        'Wiley/Blackwell':'Wiley',
        'Wiley-Blackwell, John Wiley & Sons':'Wiley',
        'Wiley-VCH':'Wiley',
        'Wolters Kluwer Health':'Wolters Kluwer',
        'Wolters Kluwer N.V./Lippinott':'Wolters Kluwer'        
    }
    for k, v in p.items():
        if k in name.upper():
            return v
    return name
     

In [9]:
j={
    'Acs': 'American Chemical Society',
    'Acs (Amercian Chemical Society) Publications': 'American Chemical Society',
    'Asm': 'American Society for Microbiology',
    'Biomed Central Limited': 'BioMed Central',
    'Biomed Central Ltd':'BioMed Central',
    'Bmj':'British Medical Journal',
    'Bmj Group':'British Medical Journal',
    'Bmj Publishing Group':'British Medical Journal',
    'Bmj Publishing Group Ltd':'British Medical Journal',
    'Cadmus':'Cadmus Journal Services',
    'Cambridge Journals':'Cambridge University Press',
    'Cambridge Uni Press':'Cambridge University Press',
    'Cambridge Univ Press':'Cambridge University Press',
    'Cenveo Publisher Services/ASM JV1': 'Cenveo Publisher Services',
    'Cold Spring Habour Press':'Cold Spring Harbor Laboratory Press',
    'Cold Spring Harbor Press':'Cold Spring Harbor Laboratory Press',
    'Cold Spring Harbor Publications':'Cold Spring Harbor Laboratory Press',
    'Dartmouth Journals':'Dartmouth Journal Services',
    'Elseveier Science':'Elsevier',
    'Faseb':'Federation of American Societies for Experimental Biology',
    'Federation of American Societies for Experimental Biology (FASEB)':'Federation of American Societies for Experimental Biology',
    'Oup':'Oxford University Press',
    'Oxford Univ Pres':'Oxford University Press',
    'Oxford Journals':'Oxford University Press',
    'Oxford Journals (OUP)':'Oxford University Press',
    'Plos':'Public Library of Science',
    'Plos (Public Library of Science)':'Public Library of Science',
    'Royal Society': 'Royal Society of Chemistry',
    'Rsc':'Royal Society of Chemistry',
    'Rsc Publishing':'Royal Society of Chemistry',
    'Aids':'Journal of Acquired Immune Deficiency Syndromes',
    'Aids Journal':'Journal of Acquired Immune Deficiency Syndromes',
    'Aids UK':'Journal of Acquired Immune Deficiency Syndromes',
    'Jaids Journal of Acquired Immune Deficiency Syndromes':'Journal of Acquired Immune Deficiency Syndromes',
    'Journal of Acquired Immune Deficiency Syndroms (JAIDS)':'Journal of Acquired Immune Deficiency Syndromes',
    'Journal of AIDS':'Journal of Acquired Immune Deficiency Syndromes',
    'Plos 1':'Plos One',
    'Neuroimage':'NeuroImage'
}
def journal_name(string):
    string = str(string)
    string = string.lower()
    result = string.title()
    if result in j:
        result = j.get(result)
    return result

New_Journal = pd.DataFrame({'Journal title':trust['Journal title'].apply(journal_name)})

In [10]:
publisher_new = pd.DataFrame({'Publisher': df['Publisher'].apply(publisher)})

In [22]:
#Merge the cleaned costs and cleaned publishers into the dataframe
df = pd.merge(df, New_Cost, left_index=True, right_index=True)
#df = pd.merge(df, df3, left_index=True, right_index=True)
new_df['Cost'] = New_Cost['Cost']
new_df['Publisher'] = publisher_new['Publisher']
new_df['Journal title'] = New_Journal['Journal title']
new_df['Article title'] = df['Article title']
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,Cost_x,Cost_y,Cost_x,Cost_y,Cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00,0.00,0.00,0.00,0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04,2381.04,2381.04,2381.04,2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,642.56,642.56,642.56,642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,669.64,669.64,669.64,669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88,685.88,685.88,685.88,685.88


In [26]:
#Bringing it together
new_df1 = df[['Publisher', 'Journal title','Article title','Cost']]
new_df1.head()

,Publisher,Journal title,Article title,Cost
0,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,0.00
1,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04
2,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56
3,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,669.64
4,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,685.88


In [27]:
new_df1.shape

(2127, 4)

In [28]:
new_df1.head()

,Publisher,Journal title,Article title,Cost
0,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,0.00
1,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04
2,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56
3,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,669.64
4,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,685.88


In [29]:
new_df1.describe()

,Cost
count,2127.000000
mean,2033.791457
std,6033.904800
min,0.000000
25%,1280.000000
50%,1884.010000
75%,2321.305000
max,201024.000000


Statistical Values

In [30]:
new_df1['Cost'].max()

201024.0

Article having largest cost

In [31]:
max_cost = new_df1['Cost'].max()
print(new_df1[(new_df1.Cost == max_cost)])

     Publisher       Journal title  \
1987     Wiley  Movement Disorders   

                                          Article title      Cost  
1987  Limb amputations in fixed dystonia: a form of ...  201024.0  


Article having minimum cost

In [32]:
min_cost = new_df1['Cost'].min()
print(new_df1[(new_df1.Cost == min_cost)])

  Publisher           Journal title  \
0       CUP  Psychological Medicine   

                                       Article title  Cost  
0  Reduced parahippocampal cortical thickness in ...   0.0  


Top 5 costliest article title and journal title

In [33]:
 new_df1.nlargest(5, 'Cost')

,Publisher,Journal title,Article title,Cost
1987,Wiley,Movement Disorders,Limb amputations in fixed dystonia: a form of ...,201024.0
1470,Public Library of Science,PLoS One,Reducing stock-outs of life saving Malaria Com...,192645.0
986,MacMillan,NaN,Fungal Disease in Britain and the United State...,13200.0
1619,public.service.co.uk,Public Service Review,Laboratory Science in Tropical Medicine,6000.0
800,Elsevier,The Lancet Neurology,Genetic risk factors for ischaemic stroke and ...,5760.0


Cheapest 5 article title,journal title

In [34]:
new_df1.nsmallest(5, 'Cost')

,Publisher,Journal title,Article title,Cost
0,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,0.00
243,BioMed Central Ltd,Veterinary Research,Understanding foot-and-mouth disease virus tra...,9.93
100,American Society for Nutrition,American Society for Nutrition,The association between breastfeeding and HIV ...,45.94
1469,Public Library of Science,PLoS One,How well are Malaria Maps used to design and ...,122.31
1677,Sciedu Press,Journal of Biomedical Graphics and Computing,Functional MRI demonstrates pain perception in...,135.29


Finding the mode of cost

In [160]:
new_df1['Cost'].mode()

0    2040.0
dtype: float64